In [1]:
#The aim of this project is to predict the percent chance of the outcome of a battle in Commands & Colors: Napoleonics.
#I've decided to use the following variables (columns) for each side of the conflict:
#nationality(hot encoded), # of each unit type, # of field works on side, # of castles on side, # of victory points on side, # of victory points to win, # of command cards, # of tactician cards, whether move first
#The predictor is the outcome

import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
TEST_PATH = os.path.join("datasets", "napoleonicBattles", "testSet.csv");
TRAINING_PATH = os.path.join("datasets", "napoleonicBattles", "trainingSet.csv");
TOTAL_PATH = os.path.join("datasets", "napoleonicBattles", "totalSet.csv");
TEMP_PATH = os.path.join("datasets", "napoleonicBattles", "tempSet.csv");
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
testSet = pd.read_csv(TEST_PATH);
trainingSet = pd.read_csv(TRAINING_PATH);
totalSet = pd.read_csv(TOTAL_PATH);
tempSet = pd.read_csv(TEMP_PATH);

In [3]:
trainingSet.head()

,ID,French1,British1,Portuguese1,Spanish1,Russian1,Austrian1,Prussian1,Commander1,LineInfantry1,LightInfantry1,GrenadierInfantry1,YoungGuard1,OldGuardInfantry1,MilitiaInfantry1,FootArtillery1,GuardFootArtillery1,HorseArtillery1,GuardHorseArtillery1,LightCavalry1,LightLancerCavalry1,GuardLightCavalry1,HeavyCavalry1,HeavyCuirassierCavalry1,GuardHeavyCavalry1,RifleLightInfantry1,GuardGrenadierInfantry1,RocketBattery1,GuardLightInfantry1,MilitiaLancerCavalry1,CossackCavalry1,GrenzerInfantry1,ReserveInfantry1,FieldWorks1,Castles1,VictoryPointsOnSide1,VictoryPointsToWin1,French2,British2,Portuguese2,Spanish2,Russian2,Austrian2,Prussian2,Commander2,LineInfantry2,LightInfantry2,GrenadierInfantry2,YoungGuard2,OldGuardInfantry2,MilitiaInfantry2,FootArtillery2,GuardFootArtillery2,HorseArtillery2,GuardHorseArtillery2,LightCavalry2,LightLancerCavalry2,GuardLightCavalry2,HeavyCavalry2,HeavyCuirassierCavalry2,GuardHeavyCavalry2,RifleLightInfantry2,GuardGrenadierInfantry2,RocketBattery2,GuardLightInfantry2,MilitiaLancerCavalry2,CossackCavalry2,GrenzerInfantry2,ReserveInfantry2,FieldWorks2,Castles2,VictoryPointsOnSide2,VictoryPointsToWin2,CommandCards1,CommandCards2,TacticianCards1,TacticianCards2,MoveFirst1,MoveFirst2,1Won,GuerrillaRule1,GuerrillaRule2,GuerrillaCounter1,GuerrillaCounter2,MotherRussiaRule1,MotherRussiaRule2,MotherRussiaNoSaber1,MotherRussiaNoSaber2,MotherRussiaNoCavalry1,MotherRussiaNoCavalry2,IronWillCounter1,IronWillCounter2
0,0,0,0,0,0,0,1,0,2,4,2,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,6,1,0,0,0,0,0,0,0,7,3,1,0,0,2,3,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,5,5,3,3,0.0,1.0,0.50,0,0,0,0,0,0,0,0,0,0,0,0
1,9,0,0,1,0,0,0,0,3,5,2,0,0,0,1,2,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,3,1,5,8,0,0,0,1,0,0,0,4,8,2,2,0,0,0,2,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,8,4,5,2,4,0.0,1.0,0.58,0,0,0,0,0,0,0,0,0,0,0,0
2,10,0,0,1,0,0,0,0,3,5,2,0,0,0,1,2,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,3,1,5,8,0,0,0,1,0,0,0,4,8,2,2,0,0,0,2,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,8,4,5,2,4,0.0,1.0,0.58,0,0,0,0,0,0,0,0,0,0,0,0
3,11,0,0,1,0,0,0,0,3,5,2,0,0,0,1,2,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,3,1,5,8,0,0,0,1,0,0,0,4,8,2,2,0,0,0,2,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,8,4,5,2,4,0.0,1.0,0.58,0,0,0,0,0,0,0,0,0,0,0,0
4,12,0,0,1,0,0,0,0,3,5,2,0,0,0,1,2,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,3,1,5,8,0,0,0,1,0,0,0,4,8,2,2,0,0,0,2,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,8,4,5,2,4,0.0,1.0,0.58,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
testSetVals = testSet.drop(["1Won", "ID"], axis = 1)
testSetLabels = testSet["1Won"].copy()
trainingSetVals = trainingSet.drop(["1Won", "ID"], axis = 1)
trainingSetLabels = trainingSet["1Won"].copy()
totalSetVals = totalSet.drop(["1Won", "ID"], axis = 1)
totalSetLabels = totalSet["1Won"].copy()
tempSetVals = tempSet.drop(["1Won", "ID"], axis = 1)

In [5]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
LineInfantry1_ix, LightInfantry1_ix, GrenadierInfantry1_ix, YoungGuard1_ix, OldGuardInfantry1_ix, MilitiaInfantry1_ix, FootArtillery1_ix, GuardFootArtillery1_ix, HorseArtillery1_ix, GuardHorseArtillery1_ix, LightCavalry1_ix, LightLancerCavalry1_ix, GuardLightCavalry1_ix, HeavyCavalry1_ix, HeavyCuirassierCavalry1_ix, GuardHeavyCavalry1_ix, RifleLightInfantry1_ix, GuardGrenadierInfantry1_ix, RocketBattery1_ix, GuardLightInfantry1_ix, MilitiaLancerCavalry1_ix, CossackCavalry1_ix, GrenzerInfantry1_ix, ReserveInfantry1_ix = range(9, 33)
LineInfantry2_ix, LightInfantry2_ix, GrenadierInfantry2_ix, YoungGuard2_ix, OldGuardInfantry2_ix, MilitiaInfantry2_ix, FootArtillery2_ix, GuardFootArtillery2_ix, HorseArtillery2_ix, GuardHorseArtillery2_ix, LightCavalry2_ix, LightLancerCavalry2_ix, GuardLightCavalry2_ix, HeavyCavalry2_ix, HeavyCuirassierCavalry2_ix, GuardHeavyCavalry2_ix, RifleLightInfantry2_ix, GuardGrenadierInfantry2_ix, RocketBattery2_ix, GuardLightInfantry2_ix, MilitiaLancerCavalry2_ix, CossackCavalry2_ix, GrenzerInfantry2_ix, ReserveInfantry2_ix = range(45, 69)
class TotalCombatUnitsAdder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self = self
    def fit(self, X, y=None):
        return self # nothing else to do
    def transform(self, X, y=None):
        TotalCombatUnits1 = X[:, LineInfantry1_ix] + X[:, LightInfantry1_ix] + X[:, GrenadierInfantry1_ix] + X[:, YoungGuard1_ix] + X[:, OldGuardInfantry1_ix] + X[:, MilitiaInfantry1_ix] + X[:, FootArtillery1_ix] + X[:, GuardFootArtillery1_ix] + X[:, HorseArtillery1_ix] + X[:, GuardHorseArtillery1_ix] + X[:, LightCavalry1_ix] + X[:, LightLancerCavalry1_ix] + X[:, GuardLightCavalry1_ix] + X[:, HeavyCavalry1_ix] + X[:, HeavyCuirassierCavalry1_ix] + X[:, GuardHeavyCavalry1_ix] + X[:, RifleLightInfantry1_ix] + X[:, GuardGrenadierInfantry1_ix] + X[:, RocketBattery1_ix] + X[:, GuardLightInfantry1_ix] + X[:, MilitiaLancerCavalry1_ix] + X[:, CossackCavalry1_ix] + X[:, GrenzerInfantry1_ix] + X[:, ReserveInfantry1_ix]
        TotalCombatUnits2 = X[:, LineInfantry2_ix] + X[:, LightInfantry2_ix] + X[:, GrenadierInfantry2_ix] + X[:, YoungGuard2_ix] + X[:, OldGuardInfantry2_ix] + X[:, MilitiaInfantry2_ix] + X[:, FootArtillery2_ix] + X[:, GuardFootArtillery2_ix] + X[:, HorseArtillery2_ix] + X[:, GuardHorseArtillery2_ix] + X[:, LightCavalry2_ix] + X[:, LightLancerCavalry2_ix] + X[:, GuardLightCavalry2_ix] + X[:, HeavyCavalry2_ix] + X[:, HeavyCuirassierCavalry2_ix] + X[:, GuardHeavyCavalry2_ix] + X[:, RifleLightInfantry2_ix] + X[:, GuardGrenadierInfantry2_ix] + X[:, RocketBattery2_ix] + X[:, GuardLightInfantry2_ix] + X[:, MilitiaLancerCavalry2_ix] + X[:, CossackCavalry2_ix] + X[:, GrenzerInfantry2_ix] + X[:, ReserveInfantry2_ix]
        return np.c_[X, TotalCombatUnits1, TotalCombatUnits2]
TotalCombatUnitsAdderInstance = TotalCombatUnitsAdder()

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

pipeline = Pipeline([
    ('TotalCombatUnitsAdder', TotalCombatUnitsAdder()),
    ('minMax_scaler', MinMaxScaler()),
])
trainingSetValsPrepared = pipeline.fit_transform(trainingSetVals.values)
testSetValsPrepared = pipeline.fit_transform(testSetVals.values)
totalSetValsPrepared = pipeline.fit_transform(totalSetVals.values)
tempSetValsPrepared = pipeline.fit_transform(tempSetVals.values)

In [7]:
from sklearn.linear_model import LinearRegression 

lin_reg = LinearRegression()

lin_reg.fit(trainingSetValsPrepared, trainingSetLabels)

LinearRegression()

In [8]:
battles_tr = pd.DataFrame(trainingSetValsPrepared)
battles_labels = pd.DataFrame(trainingSetLabels)

In [9]:
some_data = trainingSetVals[350:355]
some_labels = battles_labels.iloc[350:355]
some_data_prepared = pipeline.transform(some_data.values)
print("Prediction:", lin_reg.predict(some_data_prepared))
#the predictions are clearly way off

Prediction: [-1.31389635e+12 -4.94361745e+12 -4.94361745e+12 -4.94361745e+12
 -4.94361745e+12]


In [10]:
from sklearn.metrics import mean_squared_error
lin_predictions = lin_reg.predict(trainingSetValsPrepared)
lin_mse = mean_squared_error(battles_labels, lin_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

0.059681092929697435

In [11]:
testSetVals = testSet.drop("1Won", axis = 1)
testSetLabels = testSet["1Won"].copy()
battles_tst = pd.DataFrame(testSetVals)
battles_tst_labels = pd.DataFrame(testSetLabels)
lin_predictions = lin_reg.predict(testSetValsPrepared)
lin_mse = mean_squared_error(battles_tst_labels, lin_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse
#this is really bad -- linear isn't going to work

374334079480.207

In [12]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(
                max_depth=999999)
forest_reg.fit(trainingSetValsPrepared, trainingSetLabels)

RandomForestRegressor(max_depth=999999)

In [13]:
print("Prediction:", forest_reg.predict(some_data_prepared))

Prediction: [0.6163 0.4752 0.4752 0.4752 0.4752]


In [14]:
some_labels
#the predictions look close enough
#they are clearly very regularized (the predictions are generally closer to 0.5)

,1Won
350,0.62
351,0.38
352,0.38
353,0.38
354,0.38


In [15]:
for_predictions = forest_reg.predict(trainingSetValsPrepared)
for_mse = mean_squared_error(battles_labels, for_predictions)
for_rmse = np.sqrt(for_mse)
for_rmse

0.004340997303188146

In [16]:
for_predictions = forest_reg.predict(testSetValsPrepared)
for_mse = mean_squared_error(battles_tst_labels, for_predictions)
for_rmse = np.sqrt(for_mse)
for_rmse
#this is pretty good

0.16653393668848854

In [17]:
#finalModel:
final_reg = RandomForestRegressor(
                max_depth=500)
final_reg.fit(totalSetValsPrepared, totalSetLabels)

RandomForestRegressor(max_depth=500)

In [18]:
final_predictions = final_reg.predict(tempSetValsPrepared)
final_predictions

array([0.3874, 0.3386])

In [19]:
final_reg.feature_importances_

array([1.23293374e-01, 3.54844116e-05, 4.96662119e-05, 1.62103644e-03,
       1.37739847e-03, 3.56782549e-02, 2.30123037e-03, 1.32595014e-02,
       1.30453213e-02, 1.03746216e-02, 9.72951082e-03, 1.08442105e-03,
       2.34836624e-04, 8.58167814e-03, 3.13150344e-02, 4.10046289e-05,
       2.75717588e-02, 0.00000000e+00, 2.05575202e-02, 5.79566476e-03,
       9.00518064e-06, 3.02942709e-02, 2.97421005e-03, 2.11381667e-02,
       4.23295003e-05, 4.38297507e-04, 0.00000000e+00, 3.45757407e-04,
       2.16956424e-04, 1.22003137e-03, 1.11450471e-02, 1.03947359e-03,
       5.89742415e-03, 1.53124907e-03, 1.05851326e-02, 5.04268093e-03,
       8.86387320e-02, 2.00804937e-05, 7.32068488e-06, 1.04762619e-03,
       1.86564454e-03, 3.37667840e-02, 3.72170409e-03, 1.57080320e-02,
       1.16321427e-02, 1.05403654e-02, 9.84798988e-03, 5.51384675e-04,
       1.02971351e-04, 9.09630887e-03, 2.98168497e-02, 3.96471064e-05,
       3.27106503e-02, 0.00000000e+00, 1.97999001e-02, 3.86567999e-03,
      

In [27]:
final_pipeline = Pipeline([
    ('dataPrepPipeline', pipeline),
    ('ml_model', final_reg),
])
final_pipeline.fit(totalSetVals.values, totalSetLabels.values)

Pipeline(steps=[('dataPrepPipeline',
                 Pipeline(steps=[('TotalCombatUnitsAdder',
                                  TotalCombatUnitsAdder()),
                                 ('minMax_scaler', MinMaxScaler())])),
                ('ml_model', RandomForestRegressor(max_depth=500))])

In [28]:
final_predictions = final_pipeline.predict(tempSetValsPrepared)
final_predictions
#having trouble putting training AND test through data prep pipeline

ValueError: X has 94 features, but MinMaxScaler is expecting 92 features as input.

In [ ]:
#try using SVR